In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score

In [2]:
item_data = pd.read_csv('item_data.csv')
view_log = pd.read_csv('view_log.csv')
train = pd.read_csv('train.csv')
test=pd.read_csv("test.csv")

In [3]:
#from our previous analysis we found that item_data and view_log have similar column i.e item_id, and view_log and train_data have user_id
viewlog_complt=pd.merge(view_log, item_data, on='item_id', how='inner')

In [4]:
viewlog_complt.columns

Index(['server_time', 'device_type', 'session_id', 'user_id', 'item_id',
       'item_price', 'category_1', 'category_2', 'category_3', 'product_type'],
      dtype='object')

In [5]:
train_f=pd.merge(viewlog_complt, train, on='user_id', how='inner')

In [6]:
train_f.shape

(11046039, 16)

In [7]:
test_f=pd.merge(test, viewlog_complt.drop_duplicates(subset=['user_id']), on='user_id', how='left')

In [8]:
test_f.isna().sum()

impression_id      0
impression_time    0
user_id            0
app_code           0
os_version         0
is_4G              0
server_time        3
device_type        3
session_id         3
item_id            3
item_price         3
category_1         3
category_2         3
category_3         3
product_type       3
dtype: int64

In [25]:
train_f.isna().sum()

server_time        0
device_type        0
session_id         0
user_id            0
item_id            0
item_price         0
category_1         0
category_2         0
category_3         0
product_type       0
impression_id      0
impression_time    0
app_code           0
os_version         0
is_4G              0
is_click           0
dtype: int64

In [9]:
final=pd.concat([train_f, test_f],0)

c:\users\praneeth\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
final.isna().sum()

app_code               0
category_1             3
category_2             3
category_3             3
device_type            3
impression_id          0
impression_time        0
is_4G                  0
is_click           90675
item_id                3
item_price             3
os_version             0
product_type           3
server_time            3
session_id             3
user_id                0
dtype: int64

In [11]:
#creating a new columns form impression_time , server_time
final.server_time=pd.to_datetime(final.server_time)
final.impression_time=pd.to_datetime(final.impression_time)
final['diff_dates']=abs(final['impression_time']-final['server_time'])
final['diff_dates']=final['diff_dates']=final['diff_dates']/np.timedelta64(1,'D')

In [12]:
final.isna().sum()

app_code               0
category_1             3
category_2             3
category_3             3
device_type            3
impression_id          0
impression_time        0
is_4G                  0
is_click           90675
item_id                3
item_price             3
os_version             0
product_type           3
server_time            3
session_id             3
user_id                0
diff_dates             3
dtype: int64

In [13]:
final['category_1'].fillna(final.category_1.mean(), inplace=True)
final['category_2'].fillna(final.category_2.mean(), inplace=True)
final['category_3'].fillna(final.category_3.mean(), inplace=True)

In [14]:
final['diff_dates'].fillna(final.diff_dates.mean(), inplace=True)
final.product_type.fillna(mode(final.product_type)[0][0], inplace=True)

In [15]:
final['item_price'].fillna(final.item_price.mean(), inplace=True)

In [17]:
final.device_type.value_counts()
#device type is hghly imbalanced it will create bias in model so we will drop it

android    11133692
iphone         2903
web             116
Name: device_type, dtype: int64

In [18]:
final.drop(['device_type'], 1, inplace=True)

In [19]:
final.columns

Index(['app_code', 'category_1', 'category_2', 'category_3', 'impression_id',
       'impression_time', 'is_4G', 'is_click', 'item_id', 'item_price',
       'os_version', 'product_type', 'server_time', 'session_id', 'user_id',
       'diff_dates'],
      dtype='object')

In [22]:
#now feature engineering i.e creating clusters for continuous variables to create categorical columns
kmeans=KMeans(n_clusters=2)
k=np.array(final.diff_dates).reshape(-1,1)
kmeans.fit(k)
final['diff_dates']=kmeans.labels_

In [26]:
kmeans=KMeans(n_clusters=2)
k=final[['category_1', 'category_2', 'category_3']]
kmeans.fit(k)
final['cat_cluster']=kmeans.labels_

In [27]:
kmeans=KMeans(n_clusters=2)
k=np.array(final.item_price).reshape(-1,1)
kmeans.fit(k)
final['item_price']=kmeans.labels_

In [29]:
#now dropping the category_1, category_2 category_3 as we need them no longer
final.drop(['category_1', 'category_2', 'category_3'], 1 , inplace=True)

In [30]:
#now concverting categorical by labelencoding
le=LabelEncoder()

In [31]:
final.dtypes

app_code                    int64
impression_id              object
impression_time    datetime64[ns]
is_4G                       int64
is_click                  float64
item_id                   float64
item_price                  int32
os_version                 object
product_type              float64
server_time        datetime64[ns]
session_id                float64
user_id                     int64
diff_dates                  int32
cat_cluster                 int32
dtype: object

In [32]:
lis=['app_code', 'os_version', 'product_type']
for x in lis:
    final[x]=le.fit_transform(final[x])

In [33]:
final.dtypes

app_code                    int64
impression_id              object
impression_time    datetime64[ns]
is_4G                       int64
is_click                  float64
item_id                   float64
item_price                  int32
os_version                  int32
product_type                int64
server_time        datetime64[ns]
session_id                float64
user_id                     int64
diff_dates                  int32
cat_cluster                 int32
dtype: object

In [35]:
train_final=final[final.is_click.isna()==False].drop(['impression_id','impression_time','item_id','server_time','session_id','user_id'],1)

In [36]:
test_final=final[final.is_click.isna()==True].drop(['is_click','impression_id','impression_time','item_id','server_time','session_id','user_id'],1)

In [37]:
#as we know this is imbalanced dataset we will perform over sampling on trian data

In [38]:
from imblearn.over_sampling import SMOTE
sme=SMOTE()

In [39]:
train_final.head()

,app_code,is_4G,is_click,item_price,os_version,product_type,diff_dates,cat_cluster
0,213,1,0.0,0,1,2290,1,0
1,213,1,0.0,0,1,2290,1,0
2,213,1,0.0,0,1,1050,1,1
3,213,1,0.0,0,1,1050,1,1
4,213,1,0.0,0,1,6773,1,1


In [40]:
test_final.head()

,app_code,is_4G,item_price,os_version,product_type,diff_dates,cat_cluster
0,127,1,0,1,1157,0,0
1,44,0,0,1,967,0,1
2,296,1,0,1,41,1,0
3,207,1,0,1,1775,1,0
4,242,1,0,1,7537,1,1


In [42]:
X_res, y_res=sme.fit_resample(train_final.drop(['is_click'],1), train_final['is_click'])

In [44]:
X_train,X_test, y_train, y_test=train_test_split(X_res, y_res, test_size=0.1)

In [48]:
dtc=DecisionTreeClassifier()

In [59]:
dtc.fit(X_res, y_res)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [60]:
dtc.score(X_test, y_test)

0.8052389754117896

In [64]:
xgbc=XGBClassifier()

In [65]:
xgbc.fit(X_res, y_res)

[19:32:12] WARNING: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [66]:
xgbc.score(X_test, y_test)

0.6421201836713175

In [69]:
roc_auc_score(xgbc.predict(X_test), y_test)

0.656543777623128